# **Differential IK as Optimization**

In lecture, we learned about how to view Differential inverse kinematics as an instance of optimization. In this exercise, we will actually define and solve these optimization problems!


**Learning Objectives**
- Understand how Differential Inverse Kinematics (IK) can be formulated as an optimization problem.
- Learn to implement controllers using Drake’s MathematicalProgram.
- Explore how constraints (joint velocity limits, position bounds, and virtual walls) can impact robotic manipulation.

**What You'll Implement**
-  The JacobianPseudoinverse controller as an optimization-based framework with joint-velocity constraints.
- A virtual wall for the manipulator through constraints in the end-effector space.

## Setup and Imports

Let us first import our standard drake functionality

In [ ]:
from typing import Callable

import numpy as np
from pydrake.all import (
    BasicVector,
    Box,
    ConstantVectorSource,
    Context,
    DiagramBuilder,
    Integrator,
    JacobianWrtVariable,
    LeafSystem,
    MathematicalProgram,
    MultibodyPlant,
    Rgba,
    RigidTransform,
    Simulator,
    SnoptSolver,
    StartMeshcat,
    ge,
    le,
)

from manipulation import running_as_notebook
from manipulation.exercises.grader import Grader
from manipulation.exercises.pick.test_differential_ik import TestDifferentialIK
from manipulation.station import LoadScenario, MakeHardwareStation
from manipulation.utils import RenderDiagram

In [ ]:
# Start the visualizer.
meshcat = StartMeshcat()

## Part 1: The Differential IK System

As we've seen in the lecture, a differential IK converts a desired end-effector velocity to joint velocities. From drake's system view of the world, how should we abstract this?

It would be reasonable to make it a `LeafSystem`, where the input is `spatial velocity_desired`, and the output is `iiwa_velocity_commanded`. We might also need an input port of `iiwa_position_measured` since our Jacobian is a function of the joint angles, and `iiwa_velocity_esimated` in case we want to use that information.

<img src="https://raw.githubusercontent.com/RussTedrake/manipulation/master/book/figures/exercises/diffik_system.png" width="700">

Since we will be going through different implementations of DifferentialIK, let's set up a DifferentialIKSystem that accepts a function to compute joint velocities upon getting the Jacobian, desired spatial velocity, and other information from the robot.

We will also abstract all the Drake workings into a wrapper `BuildAndSimulate`, which constructs and connects the following systems:
- `HardwareStation`
- `DifferentialIKSystem`
- `ConstantVectorSource` for commanding desired velocity to controller.
- `Integrator` to go from `iiwa.velocity` output of controller to `iiwa.position` input of `HardwareStation`.

The purpose of this function is to swap out different implementations of spatial-to-joint velocity mappings on the fly and simulate!

Read through the code below to get an idea of how it works.

In [ ]:
class DifferentialIKSystem(LeafSystem):
    """Wrapper system for Differential IK.
    @param plant MultibodyPlant of the simulated plant.
    @param diffik_fun function object that handles diffik. Must have the signature
           diffik_fun(J_G, V_G_desired, q_now, v_now, X_now)
    """

    def __init__(self, plant: MultibodyPlant, diffik_fun: Callable) -> None:
        self.val = True
        LeafSystem.__init__(self)
        self._plant = plant
        self._plant_context = plant.CreateDefaultContext()
        self._iiwa = plant.GetModelInstanceByName("iiwa")
        self._G = plant.GetBodyByName("body").body_frame()
        self._W = plant.world_frame()
        self._diffik_fun = diffik_fun

        self.DeclareVectorInputPort("spatial_velocity", 6)
        self.DeclareVectorInputPort("iiwa.position_measured", 7)
        self.DeclareVectorInputPort("iiwa.velocity_measured", 7)
        self.DeclareVectorOutputPort("iiwa_velocity_command", 7, self.CalcOutput)

    def CalcOutput(self, context: Context, output: BasicVector) -> None:
        V_G_desired = self.get_input_port(0).Eval(context)
        q_now = self.get_input_port(1).Eval(context)
        v_now = self.get_input_port(2).Eval(context)

        self._plant.SetPositions(self._plant_context, self._iiwa, q_now)
        J_G = self._plant.CalcJacobianSpatialVelocity(
            self._plant_context,
            JacobianWrtVariable.kQDot,
            self._G,
            [0, 0, 0],
            self._W,
            self._W,
        )
        J_G = J_G[:, 0:7]  # Ignore gripper terms

        X_now = self._plant.CalcRelativeTransform(self._plant_context, self._W, self._G)
        p_now = X_now.translation()

        # You will be defining different versions of the diffik_fn in this problem.
        v = self._diffik_fun(J_G, V_G_desired, q_now, v_now, p_now)
        output.SetFromVector(v)


def BuildAndSimulate(
    diffik_fun: Callable,
    V_d: np.ndarray,
    duration: float,
    plot_system_diagram: bool = False,
) -> None:
    builder = DiagramBuilder()

    scenario_data = """
    directives:
    - add_directives:
        file: package://manipulation/clutter.dmd.yaml
    model_drivers:
        iiwa: !IiwaDriver
            hand_model_name: wsg
        wsg: !SchunkWsgDriver {}
    """
    scenario = LoadScenario(data=scenario_data)
    station = builder.AddSystem(MakeHardwareStation(scenario, meshcat=meshcat))
    plant = station.GetSubsystemByName("plant")

    controller = builder.AddSystem(DifferentialIKSystem(plant, diffik_fun))
    integrator = builder.AddSystem(Integrator(7))
    desired_vel = builder.AddSystem(ConstantVectorSource(V_d))

    builder.Connect(controller.get_output_port(), integrator.get_input_port())
    builder.Connect(integrator.get_output_port(), station.GetInputPort("iiwa.position"))
    builder.Connect(
        station.GetOutputPort("iiwa.position_measured"),
        controller.get_input_port(1),
    )
    builder.Connect(
        station.GetOutputPort("iiwa.velocity_estimated"),
        controller.get_input_port(2),
    )
    builder.Connect(desired_vel.get_output_port(), controller.get_input_port(0))

    diagram = builder.Build()
    diagram.set_name("diagram")
    if running_as_notebook and plot_system_diagram:
        RenderDiagram(diagram)

    simulator = Simulator(diagram)
    context = simulator.get_mutable_context()
    station_context = station.GetMyContextFromRoot(context)
    station.GetInputPort("iiwa.torque").FixValue(station_context, np.zeros((7, 1)))
    station.GetInputPort("wsg.position").FixValue(station_context, [0.1])

    integrator.set_integral_value(
        integrator.GetMyMutableContextFromRoot(context),
        plant.GetPositions(
            plant.GetMyContextFromRoot(context),
            plant.GetModelInstanceByName("iiwa"),
        ),
    )

    meshcat.StartRecording()
    simulator.AdvanceTo(duration)
    meshcat.PublishRecording()

Let's just make sure we've got all the systems and connections right by hooking this up with out Baseclass controller `DifferentialIKBaseSystem` that outputs zero velocities, and looking at the diagram.

For differential IK, we'll just set it up with a function that always outputs zeroes for now.

In [ ]:
def DiffIK_Zero(
    J_G: np.ndarray,
    V_G_desired: np.ndarray,
    q_now: np.ndarray,
    v_now: np.ndarray,
    p_now: np.ndarray,
) -> np.ndarray:
    return np.zeros(7)


V_d = np.zeros(6)
BuildAndSimulate(
    DiffIK_Zero,
    V_d,
    duration=5.0 if running_as_notebook else 0.1,
    plot_system_diagram=True,
)

You should see that the manipulator just stays still since our base class keeps sending zero-velocity.

## Part 2: Differential IK through Inverse Jacobian

That was some setup to get through the basic tools, but now we're onto the main part of the set! In class, we've seen that in order to map a desired spatial velocity $V^G$ to desired joint velocities, we can use the relation

$$V^G = J^G(q) v$$

and "invert" this with a Moore-Penrose pseudoinverse to achieve differential kinematics:

$$v = [J^G(q)]^{+} V^{G}$$

We can implement this controller fairly easily:

In [ ]:
def DiffIKPseudoInverse(
    J_G: np.ndarray,
    V_G_desired: np.ndarray,
    q_now: np.ndarray,
    v_now: np.ndarray,
    p_now: np.ndarray,
) -> np.ndarray:
    v = np.linalg.pinv(J_G).dot(V_G_desired)
    return v

Let's simulate this system forward. Run the cell below. If Meshcat takes a long time to load, then you can always play back / scroll through the animation using the controls panel in Meshcat.

In [ ]:
# Corresponds to [wx, wy, wz, vx, vy, vz]
V_d = np.array([0.0, 0.0, 0.0, 0.1, 0.0, 0])
BuildAndSimulate(DiffIKPseudoInverse, V_d, duration=7.0 if running_as_notebook else 0.1)

Wow - what just happened? There are several things you should have noticed.
- The robot did its job by going 0.1m/s in the x-direction (for anyone not familiar, there is a 'world-frame' axis colored by RGB. The usual convention is that RGB corresponds respectively to XYZ axii. Note that the robot is moving in the direction of the red axis as expected)
- When the robot reached the end of its workspace, it freaked out, jerking unexpectedly.

Imagine thinking the PseudoInverse method was a good idea, implementing the controller, then running this in a real robot - something might have gone badly wrong!

## Part 3: Pseudoinverse as Optimization

Even though the method got unreliable at the limits of the kinematic workspace (i.e. under kinematic singularities), it worked reasonably well. How do we keep the benefits of the original method while improving behavior near singularities?

One idea presented in class is to stick to a certain **joint velocity limit**. This will at least prevent the manipulator from doing something crazy.


In the language of optimization, the problem we are trying to solve will look like:

$$\begin{aligned} \min_v \quad & \|\mathbf{J}(q)v - V_d\|^2_2 \\ \textrm{s.t.} \quad &  |v|\leq v_{max} \end{aligned}$$

**YOUR TASK**: Below, you must transcribe and solve this optimization problem through `MathematicalProgram`.

NOTE: The absolute value is taken element wise, so the last constraint is really saying "for all $i$, make sure none of the elements of $|v[i]|$ exceed $v_{max}[i]$. Such constraints are also called Bounding Box constraints.

HINT: We can't directly enter absolute values as a constraint in `MathematicalProgram`, but we can break down the absolute value constraint into two separate constraints.

NOTE: If you're not familiar with how to write down mathematical program, make sure you do the `intro_to_qp` exercise on the textbook first. If you're still stuck, feel free to get more practice from [this notebook tutorial](https://deepnote.com/workspace/Drake-0b3b2c53-a7ad-441b-80f8-bf8350752305/project/Tutorials-2b4fc509-aef2-417d-a40d-6071dfed9199/notebook/mathematical_program-4c4f4de7d5d3452daacf45c20b315d31), or look at the [`pydrake` documentation on `MathematicalProgram`](https://drake.mit.edu/pydrake/pydrake.solvers.html#pydrake.solvers.MathematicalProgram).

In [ ]:
def DiffIKQP(
    J_G: np.ndarray,
    V_G_desired: np.ndarray,
    q_now: np.ndarray,
    v_now: np.ndarray,
    p_now: np.ndarray,
) -> np.ndarray:
    prog = MathematicalProgram()
    v = prog.NewContinuousVariables(7, "v")
    v_max = 3.0  # do not modify

    # TODO: Add cost and constraints to prog here.

    solver = SnoptSolver()
    result = solver.Solve(prog)

    if not (result.is_success()):
        raise ValueError("Could not find the optimal solution.")

    v_solution = result.GetSolution(v)

    return v_solution

In [ ]:
def DiffIKQP(
    J_G: np.ndarray,
    V_G_desired: np.ndarray,
    q_now: np.ndarray,
    v_now: np.ndarray,
    p_now: np.ndarray,
) -> np.ndarray:
    prog = MathematicalProgram()
    v = prog.NewContinuousVariables(7, "v")
    v_max = 3.0  # do not modify

    # TODO: Add cost and constraints to prog here.
    error = J_G.dot(v) - V_G_desired
    prog.AddCost(error.dot(error))
    # prog.AddBoundingBoxConstraint(-v_max * np.ones(7), v_max * np.ones(7), v)
    prog.AddConstraint(le(v, v_max * np.ones(7)))
    prog.AddConstraint(ge(v, -v_max * np.ones(7)))

    solver = SnoptSolver()
    result = solver.Solve(prog)

    if not (result.is_success()):
        raise ValueError("Could not find the optimal solution.")

    v_solution = result.GetSolution(v)

    return v_solution

When you're ready to test out your new method, you can run the simulator to see what happens. Similar to last time, wait until everything loads on Meshcat and run the second cell below to simulate forward.

In [ ]:
V_d = np.array(
    [0.0, 0.0, 0.0, 0.1, 0.0, 0.0]
)  # Feel free to play around with different V_desired
simulator = BuildAndSimulate(
    DiffIKQP, V_d, duration=7.0 if running_as_notebook else 0.1
)

When you have implemented things correctly, you should see we behave much better near singularities now, since we're constraining joint velocities. This is nice - even if the Jacobian becomes singular, we can guard against sudden jerky movements because we've put limits.

## Part 4: Virtual Wall in End-Effector Space

In class, we saw that in addition to doing joint velocity constraints, we can also put constraints on **joint positions** and **accelerations** through first-order Euler finite difference. Given $v_{now}$ and $q_{now}$, we can find $v$ that satisfies the following program:

$$\begin{aligned} \min_v \quad & \|\mathbf{J}(q_{now})v - V_d\|^2_2 \\ \textrm{s.t.} \quad &  |v|\leq v_{max} \\  \quad & q_{min} \leq q_{now} + h v \leq q_{max} \\ \quad & a_{min} \leq (v-v_{now})/h \leq a_{max}\end{aligned}$$

All of these constraints are formulated in the joint-space. But could we also end up with a linear constraint if we try to impose a constraint in the end-effector space?

**YOUR TASK**

You must program a robot in a factory, but you would like to implement a virtual wall that the robot is not allowed to put its hand in, since you have limited space and a human will be working there, or there are some obstacles there.

To do this, you decide to write an optimization-based controller that minimizes $\|\mathbf{J}(q_{now})v - V_d\|_2^2$ and has two constraints:
1. Joint velocity constraint $|v|\leq v_{max}$
2. A constraint that the **position** of the end-effector must be inside the following box:
  - `lower_bound = [-0.3, -1.0, 0.0]`
  - `upper_bound = [0.3, 1.0, 1.0]`

You can access the current position of the robot by using `p_now` (found by running forward kinematics).

HINT: We can similarly use first-order approximations to query what the next position will be given the current position and velocities in the end-effector space.

In [ ]:
# functions for visualizing the box that the end-effector must stay inside


def show_end_effector_box() -> None:
    vspace = Box(0.6, 2.0, 1.0)
    meshcat.SetObject("/end_effector_box", vspace, Rgba(0.1, 0.5, 0.1, 0.2))
    meshcat.SetTransform("/end_effector_box", RigidTransform([0.0, 0.0, 0.5]))


def hide_end_effector_box() -> None:
    meshcat.Delete("/end_effector_box")

In [ ]:
def DiffIKQP_Wall(
    J_G: np.ndarray,
    V_G_desired: np.ndarray,
    q_now: np.ndarray,
    v_now: np.ndarray,
    p_now: np.ndarray,
) -> np.ndarray:
    prog = MathematicalProgram()
    v = prog.NewContinuousVariables(7, "joint_velocities")
    v_max = 3.0  # do not modify
    h = 4e-3  # do not modify
    lower_bound = np.array([-0.3, -1.0, 0.0])  # do not modify
    upper_bound = np.array([0.3, 1.0, 1.0])  # do not modify

    # TODO: Add cost and constraints to prog here.

    solver = SnoptSolver()
    result = solver.Solve(prog)

    if not (result.is_success()):
        raise ValueError("Could not find the optimal solution.")

    v_solution = result.GetSolution(v)
    return v_solution

In [ ]:
def DiffIKQP_Wall(
    J_G: np.ndarray,
    V_G_desired: np.ndarray,
    q_now: np.ndarray,
    v_now: np.ndarray,
    p_now: np.ndarray,
) -> np.ndarray:
    prog = MathematicalProgram()
    v = prog.NewContinuousVariables(7, "joint_velocities")
    v_max = 3.0  # do not modify
    h = 4e-3  # do not modify
    lower_bound = np.array([-0.3, -1.0, 0.0])  # do not modify
    upper_bound = np.array([0.3, 1.0, 1.0])  # do not modify

    # TODO: Add cost and constraints to prog here.
    error = J_G.dot(v) - V_G_desired
    prog.AddCost(error.dot(error))
    # prog.AddBoundingBoxConstraint(-v_max * np.ones(7), v_max * np.ones(7), v)
    prog.AddConstraint(le(v, v_max * np.ones(7)))
    prog.AddConstraint(ge(v, -v_max * np.ones(7)))
    prog.AddConstraint(le(p_now + h * J_G[3:6, :].dot(v), upper_bound))
    prog.AddConstraint(ge(p_now + h * J_G[3:6, :].dot(v), lower_bound))

    prog.AddConstraint(le(J_G[3:6, :].dot(v), (upper_bound - p_now) / h))
    prog.AddConstraint(ge(J_G[3:6, :].dot(v), (lower_bound - p_now) / h))

    solver = SnoptSolver()
    result = solver.Solve(prog)

    if not (result.is_success()):
        raise ValueError("Could not find the optimal solution.")

    v_solution = result.GetSolution(v)
    return v_solution

In [ ]:
show_end_effector_box()
V_d = np.array([0.0, 0.0, 0.0, 0.1, 0.05, 0])
simulator = BuildAndSimulate(
    DiffIKQP_Wall, V_d, duration=7.0 if running_as_notebook else 0.1
)

In [ ]:
# hide_end_effector_box() # uncomment if you want to hide the box visualization.

If you implemented this correctly, the end effector will stay within the visualized box, almost like the robot has a wall that it cannot cross! Note that constraining the position of the end effector is not the same as constraining the *entire* end-effector to stay within the box- if part of it is outside, that is okay! (It is also a worthwhile exercise to think about why it is so).

Below is our autograder where you can check the correctness of all your implementations.

In [ ]:
Grader.grade_output([TestDifferentialIK], [locals()], "results.json")
Grader.print_test_results("results.json")

## VERIFICATION IN GRADESCOPE

**Prerequisites:** You must complete ALL the TODOs above before these verification exercises will work!

**Instructions:** We've given you a set of parameters, implement the exercises that follow and upload your solutions to Gradescope


In [ ]:
J_G = np.array(
    [
        [
            0.00000000e00,
            9.02435102e-01,
            2.54531849e-02,
            -6.52856354e-01,
            7.26280879e-01,
            6.83118291e-01,
            2.12647126e-04,
        ],
        [
            0.00000000e00,
            4.30825819e-01,
            -5.33158564e-02,
            -7.57108057e-01,
            -6.34577338e-01,
            7.09128073e-01,
            -2.39276758e-01,
        ],
        [
            1.00000000e00,
            1.29333062e-17,
            9.98253252e-01,
            -2.37901464e-02,
            2.64249288e-01,
            -1.74604626e-01,
            -9.70951383e-01,
        ],
        [
            3.15610018e-01,
            1.45973504e-01,
            2.96994113e-01,
            3.84093679e-02,
            1.15833004e-01,
            -1.24517465e-01,
            -6.93889390e-18,
        ],
        [
            3.00354015e-01,
            -3.05765366e-01,
            2.91205259e-01,
            -4.60616310e-02,
            1.20243355e-01,
            1.13081997e-01,
            -8.91756287e-18,
        ],
        [
            0.00000000e00,
            -4.14217824e-01,
            7.98035139e-03,
            4.11844127e-01,
            -2.96068464e-02,
            -2.78946736e-02,
            2.19635643e-18,
        ],
    ]
)

V_G_desired = np.array([0.0, 0.0, 0.0, 0.1, 0.05, 0.0])

q_now = np.array(
    [
        -1.12538865,
        0.05911441,
        0.41443944,
        -1.24917317,
        -0.20671306,
        1.67556706,
        0.81804794,
    ]
)

v_now = np.array(
    [0.1628158, 0.09226121, 0.12800451, 0.0873851, -0.09184795, 0.05699749, 0.27648769]
)

p_now = np.array([0.30035402, -0.31561002, 0.67932255])

## Verification 1: DiffIKQP

**Question:** Using the given parameters and your `DiffIKQP` implementation, what are the optimal joint velocities for the first two joints? Give you answer to 4 decimal places.

In [ ]:
# TODO: Implement the Verificaiton Exercise

## Verification 2: DiffIKQP_Wall

**Question:** Using the given parameters and your `DiffIKQP_Wall` implementation, what are the optimal joint velocities for the first two joints? Give you answer to 4 decimal places.

In [ ]:
# TODO: Implement the Verificaiton Exercise